In [214]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas import Series, DataFrame
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
from pandas_summary import DataFrameSummary
from sklearn.preprocessing import scale, robust_scale, minmax_scale, maxabs_scale
from sklearn.preprocessing import normalize

In [215]:
path = 'C:/Users/tmznq/workspace/ml_scratch/dataset/titanic/train.csv'
path2 = 'C:/Users/tmznq/workspace/ml_scratch/dataset/titanic/test.csv'
path3 = 'C:/Users/tmznq/workspace/ml_scratch/dataset/titanic/gender_submission.csv'
train_data = pd.read_csv(path)
X_test_data = pd.read_csv(path2)
y_test_data = pd.read_csv(path3)

- 1. 데이터셋 확인 - 대부분의 캐글 데이터들은 잘 정제되어 있습니다. 하지만 가끔 null data가 존재합니다. 이를 확인하고, 향후 수정합니다.
- 2. 탐색적 데이터 분석(exploratory data analysis) - 여러 feature 들을 개별적으로 분석하고, feature 들 간의 상관관계를 확인합니다. 여러 시각화 툴을 사용하여 insight를 얻습니다.
- 3. feature engineering - 모델을 세우기에 앞서, 모델의 성능을 높일 수 있도록 feature 들을 engineering 합니다. one-hot encoding, class로 나누기, 구간으로 나누기, 텍스트 데이터 처리 등을 합니다.
- 4. model 만들기 - sklearn 을 사용해 모델을 만듭니다. 파이썬에서 머신러닝을 할 때는 sklearn 을 사용하면 수많은 알고리즘을 일관된 문법으로 사용할 수 있습니다. 물론 딥러닝을 위해 tensorflow, pytorch 등을 사용할 수 도 있습니다.
- 5. 모델 학습 및 예측 - trainset 을 가지고 모델을 학습시킨 후, testset 을 가지고 prediction 합니다.
- 6. 모델 평가 - 예측 성능이 원하는 수준인지 판단합니다. 풀려는 문제에 따라 모델을 평가하는 방식도 달라집니다. 학습된 모델이 어떤 것을 학습하였는 지 확인해봅니다.

#### 참고
[URL](https://blog.sicara.com/naive-bayes-classifier-sklearn-python-example-tips-42d100429e44)

In [225]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB

# Importing dataset
data = train_data

# Convert categorical variable to numeric
data["Sex_cleaned"]=np.where(data["Sex"]=="male",0,1)
data["Embarked_cleaned"]=np.where(data["Embarked"]=="S",0,
                                  np.where(data["Embarked"]=="C",1,
                                           np.where(data["Embarked"]=="Q",2,3)
                                          )
                                 )
# Cleaning dataset of NaN

# data=data[[
#     "Survived",
#     "Pclass",
#     "Sex_cleaned",
#     "Age",
#     "SibSp",
#     "Parch",
#     "Fare",
#     "Embarked_cleaned"
# ]].dropna(axis=0, how='any')

# replacing dataset of NaN
data.Age.fillna(value=data.Age.mean(), inplace=True)
data.Fare.fillna(value=data.Fare.mean(), inplace=True)

# Feature engineering
data[['Age', 'Fare']] = minmax_scale(data[['Age', 'Fare']])

In [238]:
# prior
sur_rate = data['Survived'].value_counts(normalize=True)

In [228]:
# Split dataset in training and test datasets
X_train, X_test = train_test_split(data, test_size=0.3, random_state=123)

In [233]:
# Naive Bayes
# Instantiate the classifier
gnb = GaussianNB()
mnb = MultinomialNB()
bnb = BernoulliNB()
models = [gnb, mnb, bnb]

used_features =[
    "Pclass",
    "Sex_cleaned",
    "Age",
    "SibSp",
    "Parch",
    "Fare",
    "Embarked_cleaned"
]

# Train classifier
for model in models:
    model.fit(
        X_train[used_features].values,
        X_train["Survived"]
    )
    y_pred = model.predict(X_test[used_features])

    # Print results
    print("{}: Number of mislabeled points out of a total {} points : {}\n performance {:05.2f}%\n"
          .format(model,
              X_test.shape[0],
              (X_test["Survived"] != y_pred).sum(),
              100*(1-(X_test["Survived"] != y_pred).sum()/X_test.shape[0])
    ))

GaussianNB(priors=None, var_smoothing=1e-09): Number of mislabeled points out of a total 268 points : 52
 performance 80.60%

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True): Number of mislabeled points out of a total 268 points : 50
 performance 81.34%

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True): Number of mislabeled points out of a total 268 points : 55
 performance 79.48%

